# ランダムウォーク

千鳥足で動く酔っ払いの動きをモデル化してみよう（酔歩；ランダムウォーク（random walk））。いま、一次元方向のみ（数直線上を右か左）について考えるとする。理想的な完全なる酔っ払いは、現在いる場所も分かっていないし、前にどちらへ動いたかかも覚えていないので、次の一歩で右に動くか左に動くかは完全にランダムである。よって、この酔っ払いは、次の一歩によって、現在位置から確率 $1/2$ で数直線上を $+1$ だけ動き、確率 $1/2$ で数直線上を $-1$ だけ動くとする。$n$ ステップ後の酔っ払いの変位を $x_n$ とすると、漸化式として

$$
x_{n+1} = x_n + \eta_n ,
\tag{1}
$$

と書ける。ただし、$\eta_i$ はすべての $i$ について、

$$
P(\eta_i = +1) = P(\eta_i = -1) = \frac{1}{2} ,
\tag{2}
$$

のような確率で $+1$ か $-1$ となる確率変数である[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)。$i \ne j$ に対し、$\eta_i$ と $\eta_j$ は独立である。簡単のために、$n=0$ での初期変位を

$$
x_0 = 0 ,
\tag{3}
$$

としよう。

## ランダムウォークの軌跡

まずは酔っ払い（以降、横文字で格好よくランダムウォーカーと呼ぼう）がどのように動くのか、ランダムウォークの軌跡（trajectory）を、各ステップでの変位を折れ線グラフとして描くことで図示してみよう[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)。

In [ ]:
# 一次元ランダムウォークの軌跡を描く

%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 10  # ステップ数

xpoints = []  # 各ステップでの変位を保存するリスト

# 初期変位
x = 0
xpoints.append(x)

for _ in range(n):
    # 次のステップでの変位を計算
    # 確率1/2で+1、確率1/2で-1だけ動く
    if random.random() < 0.5:
        x += 1
    else:
        x -= 1
        
    # 後でグラフにしたいので、変位を保存
    xpoints.append(x)

# 最終的な変位を表示
print(f'x_{n} = {x}')

# 軌跡を折れ線グラフとして描く
fig, ax = plt.subplots()
ax.plot(xpoints)  # 一つだけリストを渡すと縦軸に使われる（横軸は0, 1, 2, ...）
ax.grid()
ax.set_ylim(-8, 8)  # 縦軸の範囲を固定（でないと、毎回変わってしまうので）
ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$x_n$')
plt.show()

ランダムであるので、当然、実行ごとに軌跡は変わるはずである。

## 練習問題

(1) ステップ数を 100 や 1000 にして実行してみよう。その際、縦軸の範囲を $[-8, 8]$ から、それぞれ $[-25, 25]$、$[-80, 80]$ 程度に広げるとよい。

## ランダムウォークの変位の分布

ランダムウォークを実行するごとに $n$ ステップ後の変位は異なるが、どの分布はどうなるのだろうか？ランダムウォークを繰り返し、$n$ ステップ後の変位だけに注目して、そのヒストグラムを作ってみよう。

In [ ]:
# nステップ後の変位のヒストグラムを描く

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba
import random
import statistics

n = 100     # ステップ数
m = 100000  # 試行回数

@numba.jit
def walk(n):  # nステップ後の変位を返す
    x = 0
    for _ in range(n):
        if random.random() < 0.5:
            x += 1
        else:
            x -= 1
    return x

# m回ランダムウォークを行う
data = [walk(n) for _ in range(m)]

# 平均と標準偏差
print(f'平均 = {statistics.mean(data)}')
print(f'標準偏差 = {statistics.stdev(data)}')

# ヒストグラム
fig, ax = plt.subplots()
ax.hist(data, bins=np.arange(-40.5, 41), rwidth=0.8)
ax.grid()
ax.set_xlabel(fr'$x_{{{n}}}$')
ax.set_ylabel('number of events')
plt.show()

ここでは、新たに [Numba](http://numba.pydata.org/) というライブラリを使っており、`@numba.jit` というデコレータ（第6回参照）を `walk` 関数に対して適用している。ざっくりと言うと、`@numba.jit` は（ある適当な制約の下に）関数を高速化するという働きをする（制約を満たしていないと、エラーや警告が出ることがある）。特に次の練習問題でステップ数を増やした際に、`@numba.jit` をコメントアウトして無効化すると、プログラムの実行速度が遅くなることが実感できるだろう。

## 練習問題

(2) ステップ数を 400 や 900 にして実行してみよう。その際、ヒストグラムの範囲は $[-40, 40]$ から、それぞれ $[-80, 80]$、$[-120, 120]$ 程度に広げるとよい。何か気づいたことはあるだろうか？

## ランダムウォークの平均二乗変位

ランダムウォーカーが右に動くか左に動くかは等確率であるから、$n$ ステップ後の変位の分布は左右対称であり、よって平均を取ると $0$ となる。しかし、原点からの距離の平均は $0$ ではなく、上で見た通り変位は $0$ でない分散を持っている。いま、$n$ ステップ後の平均変位 $\overline{x_n}$ は $0$ なのだから、分散 $\sigma_n^2$ は平均二乗変位 $\overline{x_n^2}$ に等しい。今度は、ステップ数 $n$ を変化させて、それとともに平均二乗変位 $\overline{x_n^2}$ がどのように変わるか調べてみよう。

In [ ]:
# ランダムウォークのステップ数と平均二乗変位の関係をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba
import random
import statistics

@numba.jit
def walk(n):  # nステップ後の変位を返す（前の例題の walk 関数と同じ）
    x = 0
    for _ in range(n):
        if random.random() < 0.5:
            x += 1
        else:
            x -= 1
    return x

@np.vectorize
def calc_x2(n, m):  # nステップ後の平均二乗変位を返す（m回の平均をとる）
    data = [walk(n) ** 2 for _ in range(m)]
    return statistics.mean(data)

# nを10、30、40、...、100とし、nの関数として平均二乗変位をプロットする
# 試行回数はそれぞれ100000

n = range(10, 101, 10)
x2 = calc_x2(n, 100000)

fig, ax = plt.subplots()
ax.plot(n, x2, 'o')
ax.grid()
ax.set_xlim(0)
ax.set_ylim(0)
ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$\overline{x_n^2}$')

# 比較のための直線を追加
n = np.linspace(0, 100)
x2 = n
ax.plot(n, x2)

plt.show()

$\overline{x_n^2} = n$ という関係式が見てとれるだろうか[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)？

## ランダムウォークの再帰性

ランダムウォーカーは、平均的には時間とともに原点から離れていき、その平均二乗変位はステップ数に比例するということを上で見た。では、一度原点から離れたランダムウォーカーは、もう原点に戻ってくることはないのだろうか？否、ランダムウォーカーの動きはランダムなのだから、例えば、あるステップ数の後に $x = 10$ の位置にいたとしても、極端な話、10 回連続して左に動けば原点に戻ることができる。

それでは、ランダムウォーカーが（無限に長い間）待っていれば原点に返ってくる確率はどのくらいになるのだろうか？

この確率を計算機実験で直接求めるのは難しい。代わりに、$n$ ステップ後に原点にいる確率 $P(x_n = 0)$ を、$n$ の関数として求めてみよう。すぐにわかるように $n$ が奇数のときはこの確率は $0$ である（奇数ステップ後には変位は奇数である）ので、$n$ が偶数の時だけが問題である。

In [ ]:
# ランダムウォークのステップ数と原点にいる確率の関係をグラフにする

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba
import random

@numba.jit
def walk(n):  # nステップ後に原点にいればTrueを返す（前の例題の walk 関数と意味が違うことに注意）
    x = 0
    for _ in range(n):
        if random.random() < 0.5:
            x += 1
        else:
            x -= 1
    return x == 0

@np.vectorize
def calc_p(n, m):  # nステップ後に原点にいる確率を求める（m回試行する）
    count = 0
    for _ in range(m):
        if walk(n):
            count += 1
    return count / m

# nを10、30、40、...、100とし、nの関数として確率をプロットする
# 試行回数はそれぞれ100000

n = range(10, 101, 10)
p = calc_p(n, 100000)

fig, ax = plt.subplots()
ax.plot(n, p, 'o')
ax.grid()
ax.set_xlim(0)
ax.set_ylim(0)
ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$P(x_n = 0)$')

# 比較のための曲線を追加
n = np.linspace(5, 100)
p = 1 / (np.pi * n / 2) ** 0.5
ax.plot(n, p)

plt.show()

上のグラフより、

$$
P(x_{2n} = 0) \simeq \frac{1}{\sqrt{\pi n}} ,
\tag{4}
$$

という関係が見て取れるはずである（この近似式は $n$ が十分に大きいときに正しい）[<sup id="cite_ref-5">[5]</sup>](#cite_note-5)。

さて、ランダムウォーカーが（無限に長い間）待っていればいつかは原点に返ってくる確率であるが、この確率を $\rho$ と書くと、実は

$$
\sum_{n=0}^\infty P(x_n = 0) = \frac{1}{1-\rho} ,
\tag{5}
$$

であることが示せる。この式の意味するところは、 $\rho < 1$ であれば左辺は有限である。つまり、左辺が無限大に発散すれば、$\rho = 1$ となる。いま、式 (4) より[<sup id="cite_ref-6">[6]</sup>](#cite_note-6)、

$$
\sum_{n=0}^\infty P(x_n = 0) \sim \sum_{n=n_0}^\infty \frac{1}{\sqrt{n}} = \infty ,
\tag{6}
$$

となるので、$\rho = 1$ である。つまり、（無限に長い間）ずっと待っていればランダムウォーカーは必ず原点に戻ってくる[<sup id="cite_ref-7">[7]</sup>](#cite_note-7)。このようなランダムウォークを再帰的（recurrent）であるという[<sup id="cite_ref-8">[8]</sup>](#cite_note-8)。

## 二次元ランダムウォーク

これまで、一次元でのランダムウォークを考えてきたが、これは二次元に拡張することができる。つまり、二次元座標において、1 ステップで
- 確率 1/4 で東に 1 動く（$x$ 座標を $+1$）、
- 確率 1/4 で西に 1 動く（$x$ 座標を $-1$）、
- 確率 1/4 で北に 1 動く（$y$ 座標を $+1$）、
- 確率 1/4 で南に 1 動く（$y$ 座標を $-1$）、

とするのである。出発点は原点（$x = 0$、$y = 0$）とする。この二次元ランダムウォークの軌跡を、各ステップでの $x$座標、$y$ 座標を使って折れ線グラフとして描くことで図示してみよう。

In [ ]:
# 二次元ランダムウォークの軌跡を描く

%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 100000  # ステップ数

xpoints = []  # 各ステップでの変位を保存するリスト
ypoints = []

# 初期位置
x = 0
y = 0
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 次のステップでの変位を計算
    # 確率1/4でそれぞれ東、西、北、南に動く
    r = random.random()
    if r < 0.25:
        x += 1
    elif r < 0.5:
        x -= 1
    elif r < 0.75:
        y += 1
    else:
        y -= 1
        
    # 後でグラフにしたいので、変位を保存
    xpoints.append(x)
    ypoints.append(y)

# 最終的な変位を表示
print(f'x(n = {n}) = {x}')
print(f'y(n = {n}) = {y}')

# 軌跡を折れ線グラフとして描く
fig, ax = plt.subplots()
ax.plot(xpoints, ypoints)
ax.grid()
ax.set_aspect('equal')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
plt.show()

## 練習問題

(3) ステップ数を（大幅に）増やして何回か実行してみよう。ステップ数を十万～百万程度に増やすと、（全体のスケールから見ると 1 ステップは十分に小さいので）もはや軌跡はギザギザではなく、なめらかで不規則な曲線として見えるだろう。

## 多次元ランダムウォーク

同様にして、三次元以上でのランダムウォークを考えることができる。次元数を $d$ とすると、1 ステップで確率 $1/(2d)$ によって各座標軸に沿って $+1$、もしくは $-1$ の方向に動くのである（三次元であれば、確率 $1/6$ で、上、下、左、右、前、後ろのいずれかの方向に動く）。一次元の場合と同じように、ランダムウォーカーは平均的には原点にいるが、平均二乗変位は $n$ となる：

$$
\overline{\vec{x}_n} = \vec{0} ,
\qquad
\overline{\vec{x}_n^2} = n .
\tag{7}
$$

再帰性についてはどうだろうか？ 結果だけを書くと、$2n$ ステップ後に原点にいる確率について、$n$ が大きいときに

$$
P(\vec{x}_{2n} = \vec{0}) \sim \frac{1}{n^{d/2}} ,
\tag{8}
$$

のように振る舞うことが示せる。これより、

$$
\sum_{n=0}^\infty P(\vec{x}_n = \vec{0}) = \begin{cases}
\infty, & d = 1, 2 ,\\
\text{有限の値}, & d \ge 3 ,
\end{cases}
\tag{9}
$$

となる。よって、一次元と二次元では再帰的であるが、三次元以上では再帰的ではない[<sup id="cite_ref-9">[9]</sup>](#cite_note-9)。

## 演習課題

(1) 今回学んだ一次元ランダムウォークでの平均二乗変位はステップ数 $n$ に比例した。このことは、ランダムウォーカーの動き方を少々変えても変わらない。$i$ ステップでの増分 $\eta_i$ に関して、次の (a) から (e) のうちの一つを使い[<sup id="cite_ref-10">[10]</sup>](#cite_note-10)（どれを選ぶべきかは**授業中に指定する**）、一次元ランダムウォークのステップ数 $n$ と平均二乗変位 $\overline{x_n^2}$ の関係をグラフにせよ。その際に、比較のための直線 $\overline{x_n^2} = c n$（$c$ は比例係数）も、
```python
# 比較のための直線を追加
c = 10  # この値は例である。うまく選ぶこと。
n = np.linspace(0, 100)
x2 = c * n
ax.plot(n, x2)
```

のようにして描け[<sup id="cite_ref-11">[11]</sup>](#cite_note-11)。

(a) $\eta_i$ は、$[-\sqrt{3}, \sqrt{3}]$ の範囲に一様分布する一様乱数。

(b) $\eta_i$ は、$[-\sqrt{6}, \sqrt{6}]$ の範囲に一様分布する一様乱数。

(c) $\eta_i$ は平均 $0$、標準偏差 $\sqrt{3}$ の正規分布に従う正規乱数。

(d) $\eta_i$ は平均 $0$、標準偏差 $2$ の正規分布に従う正規乱数。

(e) $\eta_i$ は、確率 $1/4$ でそれぞれ $+3$、$+1$、$-1$、$-3$ となるような乱数。

In [ ]:
# 課題解答10.1  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba
import random
import statistics




## 発展課題

余裕があればやってみてください。

(2) 1 ステップでの増分 $\eta_i$ が平均 $0$、標準偏差 $1$ の正規分布に従う正規乱数であるような一次元ランダムウォークを考えよう。$n$ ステップのランダムウォーク中の各変位（$x_0, x_1, x_2, \dots, x_{n-1}, x_n$）の中で、最小値を $x_\text{min}$、最大値を$x_\text{max}$ としたとき、

$$
x_\text{min} = x_0 , \qquad
x_\text{max} = x_n ,
\tag{10}
$$

となるような確率を $P_n$ とする[<sup id="cite_ref-12">[12]</sup>](#cite_note-12)。$n$ と $P_n$ の関係をグラフにせよ。その際に、比較する曲線として、

$$
P_n = \frac{1}{2n} ,
\tag{11}
$$

も描け。

In [ ]:
# 課題解答10.2  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba
import random




# 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
この場合、一次元対称単純ランダムウォーク（one-dimensional simple symmetric random walk）と呼ばれる。

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
ここでは
```python
    if random.random() < 0.5:
        x += 1
    else:
        x -= 1
```
のように $1$ ステップの動きを実装している。このような書き方（を少し一般化したもの）は次回も出てくるが、これは [random モジュール](https://docs.python.org/ja/3/library/random.html)の [choice 関数](https://docs.python.org/ja/3/library/random.html#random.choice)を用いて
```python
    x += random.choice([-1, 1])
```
と書いてもよい。ただし、生成されるシーケンスは違うものとなる。また、Numba も現時点ではサポートしていないようである（これに関しては `numpy.random.choice` を使うという手がある）。

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
式 (2) の確率変数 $\eta_i$ の定義から $E[\eta_i] = 0$ と $E[\eta_i^2] = 1$ で、また $i \ne j$ に対して $E[\eta_i \eta_j] = E[\eta_i] E[\eta_j] = 0$ なので、すぐに $E[x_n] = 0$、$E[x_n^2] = n$ であると計算できる。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
いま考えているランダムウォークは、ブラウン運動の簡単なモデルの一つである。20世紀初頭、原子や分子の実在について科学者の多数派は懐疑的であった。1905年、アルベルト・アインシュタインは、ブラウン運動を媒質分子の不規則な熱運動によって説明する論文を発表する：

- A. Einstein, *Über die von der molekularkinetischen Theorie der Wärme geforderte Bewegung von in ruhenden Flüssigkeiten suspendierten Teilchen*, [*Ann. Phys.* 322 (1905) 549-560](https://doi.org/10.1002/andp.19053220806).

コロイド粒子は絶えずあらゆる方向からの媒質分子の衝突を受け不規則に運動するが、確率論的にその平均二乗変位を論じることはできる。ある一方向を $x$ 軸と取ると、その方向の平均二乗変位は $\overline{x^2} = 2 D t$ のように観察時間 $t$ に比例する（今回のランダムウォークの $\overline{x_n^2} \propto n$ に対応する）。アインシュタインは、もし分子運動論が正しいとすれば、拡散係数 $D$ が

$$
D = \frac{R T}{N_A} \frac{1}{6\pi \eta a} ,
$$

となることを導き（$R$ は気体定数、$T$ は絶対温度、$N_A$ はアボガドロ定数、$\eta$ は液体の粘性係数、$a$ はコロイド粒子の半径）、1 分間にコロイド粒子が 6 µm 程度動き得ると見積もった。これは光学顕微鏡で観察可能な大きさである。実際、ジャン・ペランによって、ブラウン運動をしているコロイド粒子の平均二乗変位が時間に比例することが確かめられた（1908 年）。人類は原子・分子の存在を顕微鏡の中に「見る」ことができるようになったのである。

<span id="cite_note-5">5.</span> [^](#cite_ref-5)
$2n$ ステップ後に原点にいるためには、$2n$ ステップのうち $n$ ステップだけ右に動けばよい（あとの $n$ ステップは左である）。すべての場合の $2^{2n}$ 通りに対し、これは ${}_{2n}C_n$ 通りあるから、

$$
P(x_{2n} = 0) = \frac{{}_{2n}C_n}{2^{2n}}
= \frac{1}{\sqrt{\pi n}} \Biggl( 1 + \mathcal{O}\biggl(\frac{1}{n}\biggr) \Biggr) ,
$$

となる。

<span id="cite_note-6">6.</span> [^](#cite_ref-6)
級数が発散するかどうかが問題なので、$n$ が大きいところの振る舞いに注目しよう。

<span id="cite_note-7">7.</span> [^](#cite_ref-7)
無限に長い間待っていると必ず帰ってくるが、帰ってくるまでにかかるステップ数の期待値も無限大となる。

<span id="cite_note-8">8.</span> [^](#cite_ref-8)
これに対し、無限に長い間待っていても原点に返ってこない場合が有限の確率で起こるとき、非再帰的もしくは推移的（transient）であるという。

<span id="cite_note-9">9.</span> [^](#cite_ref-9)
このことを指して、角谷静夫は "A drunk man will eventually find his way home, but a drunk bird may get lost forever." と言ったそうだ。

<span id="cite_note-10">10.</span> [^](#cite_ref-10)
一様乱数は [random モジュール](https://docs.python.org/ja/3/library/random.html)の [uniform 関数](https://docs.python.org/ja/3/library/random.html#random.uniform)、正規乱数は [gauss 関数](https://docs.python.org/ja/3/library/random.html#random.gauss)を使えばよい。(e) のような、四つの値を等確率で選ぶような乱数は、今回の例題の中にあるはず。

<span id="cite_note-11">11.</span> [^](#cite_ref-11)
ヒント：$c$ は一桁の整数（つまり、1 から 9 のどれか）となる（ように問題を設定してある）。

<span id="cite_note-12">12.</span> [^](#cite_ref-12)
元ネタはこれ：

- Francesco Mori, Satya N. Majumdar, and Grégory Schehr, *Distribution of the time between maximum and minimum of random walks*, [*Phys. Rev.* **E** 101 (2020) 052111](https://doi.org/10.1103/PhysRevE.101.052111), [arXiv:2002.12352 [cond-mat.stat-mech]](https://arxiv.org/abs/2002.12352).

式 (11) はすべての $n$ $(\ge 1)$ に対して、また、あらゆる連続で対称な確率密度関数に対して成り立つ（という**推測**が書いてある）。